In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii


### Useful functions

## Table2.txt
read table2.txt and calculate a dictionary of positions for each field

In [2]:
tab2col = ['yr','mon','d','h','min','s','mjd','RAh','RAm','RAs','DEd','DEm','DEs','ExpID','FieldID','Other','PID','Filter','Exp']
file = './catdata/Table2.txt'
tab2df = pd.read_csv(file, header=0, index_col=False, names=tab2col,delim_whitespace=True)


In [3]:
tab2df = tab2df[['RAh','RAm','RAs','DEd','DEm','DEs','FieldID']]

In [4]:
tab2df

,RAh,RAm,RAs,DEd,DEm,DEs,FieldID
0,1,33,51.00,30,39,36.8,m026
1,1,33,50.99,30,39,37.0,m026
2,1,33,50.99,30,39,36.9,m026
3,1,33,51.00,30,39,37.0,m026
4,1,33,51.00,30,39,36.9,m026
...,...,...,...,...,...,...,...
2979,0,35,28.02,37,15,47.1,m167
2980,0,50,18.89,37,15,47.7,m166
2981,0,50,19.31,37,15,32.8,m166
2982,0,50,20.15,37,15,42.8,m166


In [5]:
fields = set(tab2df['FieldID'])

In [7]:
field_positions = {}
for f in fields:
    d = tab2df[tab2df['FieldID']==f].iloc[0] # take a single entry as the RA, Dec values are very similar
    RA = 15*(d[0]+d[1]/60+d[2]/3600)
    Dec = d[3]+d[4]/60+d[5]/3600
    field_positions[f] = (RA,Dec)

In [9]:
fields

{'m001',
 'm002',
 'm003',
 'm004',
 'm005',
 'm006',
 'm007',
 'm008',
 'm009',
 'm010',
 'm011',
 'm012',
 'm013',
 'm014',
 'm015',
 'm016',
 'm017',
 'm018',
 'm019',
 'm020',
 'm021',
 'm022',
 'm023',
 'm024',
 'm025',
 'm026',
 'm027',
 'm028',
 'm029',
 'm030',
 'm031',
 'm032',
 'm033',
 'm034',
 'm035',
 'm036',
 'm037',
 'm038',
 'm039',
 'm040',
 'm041',
 'm042',
 'm043',
 'm044',
 'm045',
 'm046',
 'm047',
 'm048',
 'm049',
 'm050',
 'm051',
 'm052',
 'm053',
 'm054',
 'm055',
 'm056',
 'm057',
 'm058',
 'm059',
 'm060',
 'm061',
 'm062',
 'm063',
 'm064',
 'm065',
 'm066',
 'm067',
 'm068',
 'm069',
 'm070',
 'm071',
 'm072',
 'm073',
 'm074',
 'm075',
 'm076',
 'm077',
 'm078',
 'm079',
 'm080',
 'm081',
 'm082',
 'm083',
 'm084',
 'm085',
 'm086',
 'm087',
 'm088',
 'm089',
 'm090',
 'm091',
 'm092',
 'm093',
 'm094',
 'm095',
 'm096',
 'm097',
 'm098',
 'm099',
 'm100',
 'm101',
 'm102',
 'm103',
 'm104',
 'm105',
 'm106',
 'm107',
 'm108',
 'm109',
 'm110',
 'm111',
 

In [8]:
field_positions

{'m036': (13.481666666666667, 31.325305555555556),
 'm014': (23.462083333333336, 29.66),
 'm335': (4.10425, 45.09580555555556),
 'm402': (8.124333333333333, 49.13977777777778),
 'm346': (10.900458333333333, 45.74475),
 'm058': (24.012125, 32.29),
 'm061': (8.689916666666667, 32.334833333333336),
 'm101': (8.290291666666667, 34.280388888888886),
 'm232': (12.110708333333333, 40.192249999999994),
 'm283': (22.377416666666665, 42.696861111111104),
 'm137': (2.9444166666666667, 36.12213888888889),
 'm319': (6.825708333333333, 44.16191666666666),
 'm288': (0.2980833333333333, 42.846361111111115),
 'm330': (0.151875, 44.82325),
 'm171': (20.809375, 37.70880555555556),
 'm342': (22.939083333333333, 45.534666666666666),
 'm113': (17.678541666666668, 35.08261111111111),
 'm197': (0.7802083333333334, 38.902499999999996),
 'm381': (10.914375, 47.73525),
 'm375': (13.594, 47.136361111111114),
 'm030': (22.762083333333333, 30.889972222222223),
 'm001': (24.562083333333334, 27.64),
 'm242': (19.6743

## Read in data

Import ascd files using astropy

In [ ]:
CAT = {}
cats_to_read = range(236,258) # 236->257 spans directly across M31
                              # 258->279 spans across M31, one row above
                              # 213->235 spans across M31 one row below

In [ ]:
columns = ['RA', 'Dec','iccd','xg','yg','g','dg','ig','xi','yi','i','di','ii','ia','field']
for a in cats_to_read:
    CAT[a] = ascii.read(f'./catdata/cats/m{a:03d}p.ascd', guess=False, header_start=0, names=columns, format='commented_header')
    CAT[a]['RA'] = coord.Angle(CAT[a]['RA'],u.h).deg
    CAT[a]['Dec'] = coord.Angle(CAT[a]['Dec'],u.deg).deg

In [ ]:
def read_cat(c: range) -> dict:
    columns = ['RA', 'Dec','iccd','xg','yg','g','dg','ig','xi','yi','i','di','ii','ia','field']
    CAT = {}
    for a in c:
        print(f'\nReading {a:03d}')
        CAT[a] = ascii.read(f'./catdata/cats/m{a:03d}p.ascd', guess=False, header_start=0, names=columns, format='commented_header')
        print('Converting RA/Dec')
        CAT[a]['RA'] = coord.Angle(CAT[a]['RA'],u.h).deg
        CAT[a]['Dec'] = coord.Angle(CAT[a]['Dec'],u.deg).deg
        print('Centering RA values')
        CAT[a]['RA'] = [(a+20)%360-20 for a in CAT[a]['RA']]
        print('Saving...')
        with open(f'./pickle/temp/{a:03d}.pk','wb') as f:
            pickle.dump(CAT[a],f)
    return CAT

In [ ]:
CAT201_235 = read_cat(range(201,236))
with open('./pickle/cat201_235.pk','wb') as f:
    pickle.dump(CAT201_235,f)

In [ ]:
CAT280_320 = read_cat(range(280,321))
with open('./pickle/cat280_320.pk','wb') as f:
    pickle.dump(CAT280_320,f)

In [ ]:
del CAT280_320
CAT321_360 = read_cat(range(321,361))
with open('./pickle/cat321_360.pk','wb') as f:
    pickle.dump(CAT321_360,f)

In [ ]:
del CAT321_360
CAT361_406 = read_cat(range(361,407))
with open('./pickle/cat361_406.pk','wb') as f:
    pickle.dump(CAT361_406,f)

In [ ]:
CAT = pickle.load(open('./pickle/cat213_279.pk','rb'))

# Convert data types in PAndAS DFs
Convert from float64 to float32 etc.

In [ ]:
cat_files = ['cat1_50.pk','cat51_100.pk','cat101_150.pk','cat151_200.pk','cat201_235.pk',
             'cat236_257.pk','cat258_279.pk','cat280_320.pk','cat321_360.pk','cat361_406.pk']

In [ ]:
# iterate for each column and convert number to different datatype
types = {'RA':'float32','Dec':'float32','iccd':'uint8','xg':'float32','yg':'float32','g':'float32','dg':'float32','ig':'int8',
        'xi':'float32','yi':'float32','i':'float32','di':'float32','ii':'int8','ia':'int8','field':'uint16'}
def convert_data_types(cat):
    for c in cat:
        for col in types:
            cat[c][col] = cat[c][col].astype(types[col])

In [ ]:
for file in cat_files:
    print(file)
    CAT = pickle.load(open(f'pickle/oldcat/{file}','rb'))
    print('converting...')
    convert_data_types(CAT)
    print('saving...\n')
    with open(f'pickle/temp/{file}','wb') as f:
        pickle.dump(CAT, f)

### Calculate mean field positions

In [ ]:
field_mean_pos = {}
for i in CAT:
    length = len(CAT[i]['RA'])
    # RA values span from 23h to 2h so we centre the values around 0 (converting 23h into a negative value)
    ra_centered = [(a+20)%360-20 for a in CAT[i]['RA']]
    mean_ra = sum(ra_centered)/length
    mean_dec = sum(CAT[i]['Dec'])/length
    field_mean_pos[i] = (mean_ra, mean_dec)

## Object count

Find an average object count across the range

Count number of each object

In [ ]:
count_sat = [len(CAT[c][CAT[c]['ig']==-9]) for c in CAT]
count_star2 = [len(CAT[c][CAT[c]['ig']==-2]) for c in CAT]
count_star1 = [len(CAT[c][CAT[c]['ig']==-1]) for c in CAT]
count_noise = [len(CAT[c][CAT[c]['ig']==0]) for c in CAT]
count_gal1 = [len(CAT[c][CAT[c]['ig']==1]) for c in CAT]
count_gal2 = [len(CAT[c][CAT[c]['ig']==2]) for c in CAT]

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
fig.set_dpi(120)

ax1.set_yscale('log')

ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], count_star2, c='g', label='star 2sig')
ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], count_star1, c='b', label='star 1sig')
ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], count_gal1, c='r', label='galaxy 1sig')
plt.legend(loc='upper right')
plt.xlabel('RA (deg)')
plt.ylabel('Object count')
plt.title('Object counts across M31 at Dec +41°')
plt.xlim(-5,25)
plt.gca().invert_xaxis()

## Colour
Calculate average colour of each class

Colours based on ii **and** ig guesses

In [ ]:
colours = {}
#calculate mean g-i colour of each field
for n in CAT:
    col_sat = np.mean(CAT[n]['g'][(CAT[n]['ig']==-9)&(CAT[n]['ii']==-9)] - CAT[n]['i'][(CAT[n]['ig']==-9)&(CAT[n]['ii']==-9)])
    col_2sigstar = np.mean(CAT[n]['g'][(CAT[n]['ig']==-2)&(CAT[n]['ii']==-2)] - CAT[n]['i'][(CAT[n]['ig']==-2)&(CAT[n]['ii']==-2)])
    col_1sigstar = np.mean(CAT[n]['g'][(CAT[n]['ig']==-1)&(CAT[n]['ii']==-1)] - CAT[n]['i'][(CAT[n]['ig']==-1)&(CAT[n]['ii']==-1)])
    col_1siggal  = np.mean(CAT[n]['g'][(CAT[n]['ig']==1)&(CAT[n]['ii']==1)] - CAT[n]['i'][(CAT[n]['ig']==1)&(CAT[n]['ii']==1)])
    col_2siggal = np.mean(CAT[n]['g'][(CAT[n]['ig']==2)&(CAT[n]['ii']==2)] - CAT[n]['i'][(CAT[n]['ig']==2)&(CAT[n]['ii']==2)])
    colours[n] = (col_sat,col_2sigstar,col_1sigstar,col_1siggal, col_2siggal)

Colours based **only** on ig guesses

In [ ]:
colours = {}
#calculate mean g-i colour of each field
for n in CAT:
    col_sat = np.mean(CAT[n]['g'][CAT[n]['ig']==-9] - CAT[n]['i'][CAT[n]['ig']==-9])
    col_2sigstar = np.mean(CAT[n]['g'][CAT[n]['ig']==-2] - CAT[n]['i'][CAT[n]['ig']==-2])
    col_1sigstar = np.mean(CAT[n]['g'][CAT[n]['ig']==-1] - CAT[n]['i'][CAT[n]['ig']==-1]) 
    col_1siggal  = np.mean(CAT[n]['g'][CAT[n]['ig']==1] - CAT[n]['i'][CAT[n]['ig']==1])
    col_2siggal = np.mean(CAT[n]['g'][CAT[n]['ig']==2] - CAT[n]['i'][CAT[n]['ig']==2])
    colours[n] = (col_sat,col_2sigstar,col_1sigstar,col_1siggal, col_2siggal)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
fig.set_dpi(120)

ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], [colours[n][1] for n in colours], c='orange', label='2sig star')
ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], [colours[n][2] for n in colours], c='red', label='1sig star')
ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], [colours[n][3] for n in colours], c='fuchsia', label='1sig galaxy')
#ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], [colours[n][4] for n in colours], c='violet', label='2sig galaxy')
#ax1.scatter([field_mean_pos[f][0] for f in field_mean_pos], [colours[n][0] for n in colours], c='lightgrey', label='saturated')

plt.legend(loc='upper left')
plt.xlabel('RA (deg)')
plt.ylabel('Colour index (g-i)')
#plt.ylim([-1,2])
plt.title('Average object colour across M31 at Dec +41°')
plt.ylim(0,2)
plt.gca().invert_xaxis()